**28. El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones**

es con contents, entonces, pepito edito en contents 128 contenidos dejando 101 comentarios, juanito edito 115 dejando 114 comentarios, supongamos que son los unicos que editaron >100 de los ultimos contenidos. entonces la gente que edito >100 de los ultimos contenidos deja comentarios en una tasa de (101+114)/(128+115), el porcentaje es *100 de esa tasa

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
downloaded = drive.CreateFile({'id':"1SnCVFJvTsaEPrmh-PpVGH6-toBVsD6NL"})
downloaded.GetContentFile('contents.csv') 

In [ ]:
!pip install pyarrow
texts = pd.read_csv('contents.csv')
texts.to_parquet('contents.parquet')
del texts

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('contents.parquet')
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rdd.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

Pasos:

1) Filtro por los datos que necesito.

2) Mapeo donde tendré una tupla de: (1,X). El 1 será para contabilizar luego y la X puede ser un 1 si el usuario comentó en la revisión y 0 si no.

3) Reduzco por clave(id o ip), sumando los valores de las tuplas.

Finalmente tendré:

(usuario, (total de revisiones, total de comentarios))

In [ ]:
datos = rdd.filter(lambda x: (x.revisor_id != None and x.revisor_ip == None) or (x.revisor_id == None and x.revisor_ip != None))\
                .map(lambda x: ((x.revisor_id if x.revisor_id != None else x.revisor_ip),(1, 1 if x.revisor_comment != None else 0)))\
                .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])).cache()

datos.take(5)

[('2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', (13, 13)),
 (5338508.0, (24169, 18749)),
 (3615366.0, (2905, 2667)),
 (689866.0, (831, 689)),
 (936194.0, (2299, 2109))]

**Porcentaje para los usuarios que realizaron más de 1 revisión:**

In [ ]:
100 * (datos.map(lambda x: (x[1][1])).sum() / datos.map(lambda x: (x[1][0])).sum())

88.6445912225631

**Porcentaje para los usuarios que realizaron más de 10 revisiones:**

In [ ]:
aux = datos.filter(lambda x: x[1][0] > 10)

100 * (aux.map(lambda x: (x[1][1])).sum() / aux.map(lambda x: (x[1][0])).sum())

89.85935076359488

**Porcentaje para los usuarios que realizaron más de 100 revisiones:**

In [ ]:
aux2 = aux.filter(lambda x: x[1][0] > 100)

100 * (aux2.map(lambda x: (x[1][1])).sum() / aux2.map(lambda x: (x[1][0])).sum())

90.63442644504349